In [1]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'quanta.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from apps.orders.models import Facturation, Commercial
from apps.organization.models import Organization
from django.db.models import Count, F, Func, Q, Sum, Value, Window


In [3]:
organization=Organization.objects.first()
facturations = Facturation.objects.for_organization(
        organization=organization
    ).all()

In [4]:
from apps.coverages.models import CoverageFacturation


coverage_facturations = (
        CoverageFacturation.objects.for_organization(organization=organization)
        .filter(status=CoverageFacturation.CoverageFacturationStatus.ACCEPTED)
        .all()
    )

In [5]:
from apps.services.models import ServiceFacturation


In [6]:
service_facturations = ServiceFacturation.objects.for_organization(
        organization=organization
    ).all()

In [7]:
commercials = Commercial.objects.for_organization(organization=organization).annotate(name=F("user__username")).values("id", "name").all()

In [8]:
coverage_report_by_commercial = coverage_facturations.values("commercial_id").annotate(total_facturation_price=Sum(F("coverage_facturation_items__unit_price")
                * F("coverage_facturation_items__quantity")), total_facturations=Count("id", distinct=True))


In [8]:
report_by_commercial = facturations.values("commercial_id").annotate(total_facturation_price=Sum(F("items__unit_price") * F("items__quantity")), total_facturations=Count("id", distinct=True))
coverage_report_by_commercial = coverage_facturations.values("commercial_id").annotate(total_facturation_price=Sum( F("coverage_facturation_items__unit_price")
                * F("coverage_facturation_items__quantity")), total_facturations=Count("id", distinct=True))
service_report_by_commercial = service_facturations.values("commercial_id").annotate(total_facturation_price=Sum(F("price") * F("quantity")), total_facturations=Count("id", distinct=True))

In [ ]:
coverage_report_by_commercial = coverage_facturations.values("commercial_id").annotate(total_facturation_price=Sum( F("coverage_facturation_items__unit_price")))

In [ ]:
print(report_by_commercial)
print(coverage_report_by_commercial)
print(service_report_by_commercial)

In [ ]:
1032000+326000

In [ ]:
from decimal import Decimal

In [ ]:
# Get the facturation reports for each type of facturation
voucher_reports = report_by_commercial.values("commercial_id", "total_facturation_price", "total_facturations")
reports = coverage_report_by_commercial.values("commercial_id", "total_facturation_price", "total_facturations")
service_reports = service_report_by_commercial.values("commercial_id", "total_facturation_price", "total_facturations")

# Create a dictionary to store the facturation reports by commercial ID
voucher_facturation_data = {report["commercial_id"]: report for report in voucher_reports}
facturation_data = {report["commercial_id"]: report for report in reports}
service_facturation_data = {report["commercial_id"]: report for report in service_reports}

# Initialize the list to store the final report
report_list = []

# Iterate over each commercial
for commercial in commercials:
    commercial_id = commercial["id"]
    commercial_name = commercial["name"]

    # Initialize the report dictionary for the current commercial
    report = {
        "commercial": commercial_name,
        "voucher_facturation_data": voucher_facturation_data.get(commercial_id),
        "facturation_data": facturation_data.get(commercial_id),
        "service_facturation_data": service_facturation_data.get(commercial_id),
        "total_facturation_price": Decimal(0.0),
        "total_facturations": 0,
    }

    # Update the total facturation price and total facturations for the current commercial
    if report["voucher_facturation_data"]:
        report["total_facturation_price"] += report["voucher_facturation_data"]["total_facturation_price"]
        report["total_facturations"] += report["voucher_facturation_data"]["total_facturations"]

    if report["facturation_data"]:
        report["total_facturation_price"] += report["facturation_data"]["total_facturation_price"]
        report["total_facturations"] += report["facturation_data"]["total_facturations"]

    if report["service_facturation_data"]:
        report["total_facturation_price"] += report["service_facturation_data"]["total_facturation_price"]
        report["total_facturations"] += report["service_facturation_data"]["total_facturations"]

    # Add the report to the final list
    report_list.append(report)


In [ ]:
for report in report_list:
    print(report)